# ABC for COVID modeling

Use numba for speed?

```
!pip install scikit-learn==0.19.1
!pip install astroabc
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from astroabc import ABC_class as ABC
from pyprojroot import here

In [ ]:
Array = Union[np.ndarray]

In [ ]:
sns.set_palette("muted")

## SIR implementation

In [ ]:
class SIR:
    """Simulate a deterministic SIR process."""
    def __init__(self, pop: int = 1000, beta: float = 0.2, gamma: float = 0.1):
        self.beta = beta
        self.gamma = gamma
        self.pop = pop
    
    def __repr__(self) -> str:
        return f"<SIR process(beta = {self.beta}, gamma = {self.gamma}, pop = {self.pop})>"
    
    def simulate(self, steps: int, init: int = 1) -> Tuple[Array, Array, Array]:
        St = np.zeros((steps + 1), dtype=int)
        It = np.zeros((steps + 1), dtype=int)
        Rt = np.zeros((steps + 1), dtype=int)
        It[0] = init
        St[0] = self.pop - init
        
        for i in range(1, steps + 1):
            b = self.beta * St[i-1] * It[i - 1] / self.pop
            g = self.gamma * It[i - 1]
            St[i] = max(0, St[i - 1] - b)
            Rt[i] = max(0, Rt[i - 1] + g)
            It[i] = max(0, It[i - 1] + (St[i - 1] - St[i]) - (Rt[i] - Rt[i - 1]))
        return St, It, Rt

In [ ]:
s, i, r = SIR(beta=0.44, gamma=0.2).simulate(steps=100)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(s.shape[-1]), s, '-',  label="S")
ax.plot(np.arange(i.shape[-1]), i, '--', label="I")
ax.plot(np.arange(r.shape[-1]), r, '-.', label="R")
ax.legend()
ax.set_xlabel("Time step")
ax.set_ylabel("Population section")
None

## Toy model

Let's try to fit the SIR trajectory using pointwise Euclidean distance.

In [ ]:
def dist_euclidean(a: Array, b: Array) -> float:
    """Assume a, b are like [..., len simulation]"""
    return np.sqrt(np.square(a - b).sum()) / a.shape[-1]

In [ ]:
def simulate_sir_simple(params):
    """Params are normal, convert to positive."""
    beta, gamma = np.exp(params)
    sim = SIR(beta=beta, gamma=gamma)
    out = sim.simulate(steps=100)[1]
    return out

In [ ]:
# Data to recover: beta, gamma, init
DATA = SIR(beta=0.44, gamma=0.2).simulate(steps=100)[1]

In [ ]:
# Use parameters from examples
sampler = ABC(
    nparam=2, 
    npart=1000, 
    data=DATA, 
    tlevels=[10, 1],
    niter=100, 
    priors=[
        ["normal", [-1., 0.1]],
        ["normal", [-1., 0.1]]
    ], 
    dfunc=dist_euclidean,
    verbose=1,
    adapt_t=True
)

sampler.sample(simulate_sir_simple)

In [ ]:
# Pretty good!
np.exp([-0.8005921380909115, -1.551978322835761])

In [ ]:
# Look at the final set of parameter samples
i = 27
fig, ax = plt.subplots()
ax.scatter(np.exp(sampler.theta[i,:,0]), np.exp(sampler.theta[i,:,1]), s=4, c=sampler.wgt[i])
ax.set_xlabel("gamma")
ax.set_ylabel("beta")
None

In [ ]:
# Marginals, too!
i = 27
fig, axs = plt.subplots(ncols=3, figsize=(10, 4), sharey=True, constrained_layout=True)
axs[0].hist(np.exp(sampler.theta[i,:,0]))
axs[1].hist(np.exp(sampler.theta[i,:,1]))
axs[2].hist(np.exp(sampler.theta[i,:,0] - sampler.theta[i,:,1]))
None

## NC COVID-19 test

Can we take a look at a trajectory at the state level?

In [ ]:
abt = pd.read_csv(here("data/processed/ABT_V1.csv"), low_memory=False)

In [ ]:
nc = abt.loc[abt.state_code == "NC", ["county_fip", "date", "confirmed", "acs_pop_total"]]
nc = nc.sort_values("date").reset_index(drop=True)
nc.head()

### Simple: Wake county

Start from the first observed case.

One problem I didn't think of immediately: the curve-matching approach requires knowing the start time.

In [ ]:
wake = nc.loc[nc.county_fip == 37183]
wake = wake.loc[wake.date >= wake.date.loc[wake.confirmed > 0].min()]
wake.head()

In [ ]:
def simulate_sir_wake(params):
    """Params are normal, convert to positive."""
    beta, gamma = np.exp(params)
    sim = SIR(beta=beta, gamma=gamma, pop=1046558)
    out = sim.simulate(steps=200)[1].max()
    return out

In [ ]:
def single_dist(x, y):
    dist = 2 * np.abs(x - y) / (x + y)
    return dist

In [ ]:
sampler = ABC(
    nparam=2, 
    npart=1000, 
    data=wake.confirmed.values.max(), 
    tlevels=[10, 0.01],
    niter=100, 
    priors=[
        ["normal", [-1., 0.5]],
        ["normal", [-1., 0.5]]
    ], 
    dfunc=single_dist,
    verbose=1,
    adapt_t=True
)

sampler.sample(simulate_sir_wake)

In [ ]:
np.exp(np.subtract(*[-1.02469649667087, -1.1083142591439332]))

In [ ]:
plt.hist(np.exp(sampler.theta[30,:,0]-sampler.theta[30,:,1]))

### Harder: All NC

1. Start at each county's first case
2. Build a SIR model up to the max length
3. Compute the distance between each series

In [ ]:
nc.groupby("county_fip").date[nc.confirmed > 0].min()